# Clasificación no supervisada con serie temporal NDVI de Terra/MODIS

In [ ]:
!pip install rasterio

### Cargar librerías

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-my-francodbarr')

In [ ]:
#importar librerías

import pandas as pd
import numpy as np
from datetime import datetime as dt
import geemap
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import Point
import rasterio
import random

Funciones

In [ ]:
def getCoordsSHP(gdfshpFile):
  '''
  This function takes a polygon vector layer file passed as a GeoDataFrame. Then
  it gets all the coordinates as a tuple of tuples.
  '''

  gdfiterf = gdfshpFile.iterfeatures()
  tupleFile = next(gdfiterf)['geometry']['coordinates']

  return tupleFile

### Seleccionar la colección y filtrarla

In [ ]:
#selecciona la colección MODIS MOD13Q1 NDVI
MODIS = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI') ##MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m

## establece el rango de fechas de interés
startDate = '2000-02-18'
endDate = '2022-12-31' ## la coleccion llega hasta 2023-02-02, aunque se informa que llega hasta julio 2023 (búsqueda agosto)

# filtrar por fechas
MODIS = MODIS.filterDate(startDate, endDate)

### Vector BBSS

In [ ]:
!wget https://github.com/francobarrionuevoenv21/ClusteringArroceras_Incendios_SanJavier/blob/main/Primary_data/Planicie_con_paleocauces_NW-SW_4326.geojson

In [ ]:
targetCRS = "EPSG:4326"
vectorPPCC = gpd.read_file('/content/Planicie_con_paleocauces_NW-SW_4326.geojson').to_crs(targetCRS)

In [ ]:
# El siguiente codigo recupera todas las coordenadas de un vector, y a partir de ellas
# crea una lista de tuplas. Con esta lista se genera finalmente un poligono.

tupleOfTuples01 = getCoordsSHP(vectorPPCC)

# Converting coordinates into a list of lists
listOfLists01 = [list(t[:2]) for t in tupleOfTuples01[0]]

# Coordinates to a Polygon
polygon01 = ee.Geometry.Polygon(listOfLists01)

# Polygon to a EE Feature Collection (For plotting the vector of the region)
featureCollection01 = ee.FeatureCollection(polygon01)

In [ ]:
# ver tamaño de la colección - es una colección con 528 elementos en mi área de prueba - 526 si dejo hasta diciembre 2022
MODIS

In [ ]:
# Define a function to unmask the NDVI band
def unmaskNDVI(image):
    return image.select('NDVI').unmask(-2000)

# Map the unmask function over the entire collection
MODIS_unmasked = MODIS.map(unmaskNDVI)

In [ ]:
# Convertir la colección a un stack - una imagen de NDVI por fecha
MODIS_stack = MODIS_unmasked.toBands()

MODIS_stack # ya no es una colección, es una imagen con 526 bandas en mi área de prueba

In [ ]:
#cortar con un shapefile
MODIS_stack = MODIS_stack.clipToCollection(featureCollection01)

### K-means

In [ ]:
# El kmeans de GEE necesita entrenar al menos con puntos al azar
# Te pide que le des puntos al azar para inicializar el algoritmo
# NO DAR muestras de entrenamiento

# Fuente: https://developers.google.com/earth-engine/guides/clustering

#Define a region in which to generate a sample of the input.

#primero defino las variables que piden más abajo
region = featureCollection01
scale = 250 ## este es el tamaño de pixel de MODIS en esta colección
#scale = 231.65635826395825 ## este es el tamaño de pixel de MODIS en esta colección

numPixels = 5000 # puntos de inicializacion --> no son de entrenamiento

# Make the training dataset, except for -999 (mascara)
#MODIS_randomtraining = MODIS_stack.sample(region = region, scale = scale, numPixels = numPixels)

MODIS_randomtraining = MODIS_stack.sample(region=region, scale=scale, numPixels=numPixels)

In [ ]:
# Iniciar una clasificación kmeans con N clusters

num_clusters = 50 # numero de clases
# empezar > 8 ==> David ya clasifico 8, incluyendo arroceras
kmeans = ee.Clusterer.wekaKMeans(num_clusters).train(MODIS_randomtraining)

In [ ]:
# Aplica la clasificación a la imagen
MODIS_kmeans = MODIS_stack.cluster(kmeans) # cada pixel va de 0-n_clusters-1

MODIS_kmeans = MODIS_kmeans.add(1) #sumar 1 para que no empiece en 0 el cluster numbering
# cada pixel va de 1-n_clusters

MODIS_kmeans = MODIS_kmeans.clip(featureCollection01)

In [ ]:
MODIS_kmeans # es una imagen de una banda, lo que está bien. espero que sea una clasificación.

In [ ]:
# generar mapa

# ver clasificación - en Python

Map = geemap.Map(center=(-32.06, -60.62), zoom=9)
Map.addLayer(MODIS_kmeans)

Map

In [ ]:
#https://worldbank.github.io/OpenNightLights/tutorials/mod3_7_import_export_data.html

task = ee.batch.Export.image.toDrive(
        image = MODIS_kmeans.toDouble(),
        scale=scale,
        region = featureCollection01.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
        description = 'MODIS_MOD13Q1_NDVI_kmeans50-17-04.tif',
        crs = 'EPSG:4326',
        maxPixels = 1e10,
        fileFormat = "GeoTIFF",
        folder ='earthengine'
  )

task.start()

## Resumen de las clases

#### Extraccion de los datos para su visualizacion en forma de SSTT de NDVI

In [ ]:
# https://drive.google.com/file/d/1yRZ7BXNJlEytsreXhU2A4wI_MQkPnlMt/view?usp=sharing

!gdown --id 1yRZ7BXNJlEytsreXhU2A4wI_MQkPnlMt

In [ ]:
# Load raster classification
raster_file = "/content/MODIS_MOD13Q1_NDVI_kmeans50-17-04.tif.tif"
with rasterio.open(raster_file) as src:
    classification = src.read(1)
    transform = src.transform
    num_clusters = len(np.unique(classification))

# Create vector layer with random points
num_points_per_cluster = 100 #
#num_points_per_cluster = 5 #prueba con pocos puntos

points = []
cluster_numbers = []

for cluster_id in range(1, num_clusters + 1):
    cluster_indices = np.where(classification == cluster_id)
    sample_indices = random.sample(range(len(cluster_indices[0])), min(num_points_per_cluster, len(cluster_indices[0])))

    for index in sample_indices:
        row, col = cluster_indices[0][index], cluster_indices[1][index]
        x, y = rasterio.transform.xy(transform, row, col)
        points.append(Point(x, y))
        cluster_numbers.append(cluster_id)


# Create a GeoDataFrame from the points
data = {'geometry': points, 'cluster': cluster_numbers}
gdf = gpd.GeoDataFrame(data, crs=src.crs)

# Save the GeoDataFrame to a shapefile
output_shapefile = "kmeans_random_points.shp"
gdf.to_file(output_shapefile)

print("Random points saved to:", output_shapefile)

In [ ]:
modis_collection = MODIS
#modis_collection = MODIS.filterDate("2022-01-01", "2022-01-31") #prueba con pocas imagenes


clusters = gdf["cluster"].unique()

# Create a function to extract NDVI values for a point
def get_ndvi_for_point(feature):
    point = feature.geometry()
    ndvi_values = modis_collection.getRegion(point, scale=250).getInfo()
    return ndvi_values

# Loop through each cluster and extract NDVI values for each point
ndvi_data = []

for cluster in clusters:
    cluster_gdf = gdf[gdf["cluster"] == cluster]
    for index, row in cluster_gdf.iterrows():
        point = Point(row["geometry"].x, row["geometry"].y)
        feature = ee.Feature(ee.Geometry.Point(point.x, point.y))
        ndvi_values = get_ndvi_for_point(feature)

        # Extract NDVI values for each scene
        for scene_values in ndvi_values[1:]:
            timestamp_milliseconds = int(scene_values[3])
            timestamp_seconds = timestamp_milliseconds / 1000
            dt_object = dt.utcfromtimestamp(timestamp_seconds)
            formatted_date = dt_object.strftime('%Y-%m-%d')

            scene_ndvi_values = scene_values[4:]
            ndvi_data.append([cluster, point.x, point.y, formatted_date, *scene_ndvi_values])

# Create a dataframe from the extracted data
columns = ["cluster", "longitude", "latitude", "date"] + ["NDVI" + str(i) for i in range(1, len(ndvi_data[0]) - 3)]
ndvi_df = pd.DataFrame(ndvi_data, columns=columns)

# NOTA: este paso para el Delta tarda unas 2 horas

In [ ]:
#llenar los NoData con -2000 (igual que hicimos en la imagen con unmask, ahora lo hace en la tabla)
ndvi_df = ndvi_df.fillna(-2000)

In [ ]:
# Display the dataFrame
ndvi_df

In [ ]:
# Pivot the DataFrame to have one column per date
pivot_df = ndvi_df.pivot_table(
    index=["cluster", "longitude", "latitude"],
    columns="date",
    values=["NDVI1"],  # List all the NDVI columns here
    aggfunc="first"  # You can adjust the aggregation function if needed
)

# Reset the index to make columns regular columns
pivot_df.reset_index(inplace=True)

# Display the pivoted DataFrame
pivot_df

In [ ]:
#export
pivot_df.to_csv("./kmeans_ndvi_pivot.csv", index=False)

ndvi_df.to_csv("./kmeans_ndvi.csv", index=False)